In [23]:
# importing all necessary libraries
import pickle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import (LinearRegression, Lasso, Ridge)

from sklearn.metrics import mean_squared_error 

In [27]:
# reading the parquet file with the data
data = pd.read_parquet('../data/january_2021')

In [29]:
#saving the data in csv
data.to_csv('green_tripdata.csv')

In [31]:
# importing the data
df = pd.read_csv('green_tripdata.csv')

C:\Users\PC\AppData\Local\Temp\ipykernel_20072\3769022231.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('green_tripdata.csv')


In [32]:
# the data
df.head()

,Unnamed: 0,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,...,0.5,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.00
1,1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,...,0.5,0.5,2.81,0.0,NaN,0.3,16.86,1.0,1.0,2.75
2,2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,...,0.5,0.5,1.00,0.0,NaN,0.3,8.30,1.0,1.0,0.00
3,3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,...,0.5,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.00
4,4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,...,0.0,-0.5,0.00,0.0,NaN,-0.3,-52.80,3.0,1.0,0.00


In [33]:
#data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             76518 non-null  int64  
 1   VendorID               76518 non-null  int64  
 2   lpep_pickup_datetime   76518 non-null  object 
 3   lpep_dropoff_datetime  76518 non-null  object 
 4   store_and_fwd_flag     40471 non-null  object 
 5   RatecodeID             40471 non-null  float64
 6   PULocationID           76518 non-null  int64  
 7   DOLocationID           76518 non-null  int64  
 8   passenger_count        40471 non-null  float64
 9   trip_distance          76518 non-null  float64
 10  fare_amount            76518 non-null  float64
 11  extra                  76518 non-null  float64
 12  mta_tax                76518 non-null  float64
 13  tip_amount             76518 non-null  float64
 14  tolls_amount           76518 non-null  float64
 15  eh

In [34]:
# create a column for duration
df["duration"] = pd.to_datetime(df["lpep_dropoff_datetime"]) - pd.to_datetime(df["lpep_pickup_datetime"])
df.head()

,Unnamed: 0,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,...,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.00,0 days 00:03:56
1,1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,...,0.5,2.81,0.0,NaN,0.3,16.86,1.0,1.0,2.75,0 days 00:08:45
2,2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,...,0.5,1.00,0.0,NaN,0.3,8.30,1.0,1.0,0.00,0 days 00:05:58
3,3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,...,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.00,0 days 00:07:05
4,4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,...,-0.5,0.00,0.0,NaN,-0.3,-52.80,3.0,1.0,0.00,0 days 00:00:04


In [36]:
#convert the duration column to minutes
df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)

In [37]:
df

,Unnamed: 0,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,0.000000,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,...,0.5,0.00,0.00,NaN,0.3,6.80,2.0,1.0,0.00,3.933333
1,0.000278,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,...,0.5,2.81,0.00,NaN,0.3,16.86,1.0,1.0,2.75,8.750000
2,0.000556,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,...,0.5,1.00,0.00,NaN,0.3,8.30,1.0,1.0,0.00,5.966667
3,0.000833,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,...,0.5,0.00,0.00,NaN,0.3,9.30,2.0,1.0,0.00,7.083333
4,0.001111,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,...,-0.5,0.00,0.00,NaN,-0.3,-52.80,3.0,1.0,0.00,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,21.253611,2,2021-01-31 21:38:00,2021-01-31 22:16:00,NaN,NaN,81,90,NaN,17.63,...,0.0,0.00,6.12,NaN,0.3,65.40,NaN,NaN,NaN,38.000000
76514,21.253889,2,2021-01-31 22:43:00,2021-01-31 23:21:00,NaN,NaN,35,213,NaN,18.36,...,0.0,12.20,6.12,NaN,0.3,65.28,NaN,NaN,NaN,38.000000
76515,21.254167,2,2021-01-31 22:16:00,2021-01-31 22:27:00,NaN,NaN,74,69,NaN,2.50,...,0.0,0.00,0.00,NaN,0.3,22.00,NaN,NaN,NaN,11.000000
76516,21.254444,2,2021-01-31 23:10:00,2021-01-31 23:37:00,NaN,NaN,168,215,NaN,14.48,...,0.0,0.00,6.12,NaN,0.3,58.04,NaN,NaN,NaN,27.000000


In [38]:
#get categorical and numerical data

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

categorical

['PULocationID', 'DOLocationID']

In [41]:
df[categorical].head()

,PULocationID,DOLocationID
0,43,151
1,166,239
2,41,42
3,168,75
4,265,265


In [42]:
#convert categorical data to str

df[categorical] = df[categorical].astype(str)

In [43]:
#reduce dataset to only duration between 1 and 60

df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

df.head()

,Unnamed: 0,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,0.000000,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,...,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.00,3.933333
1,0.000278,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,...,0.5,2.81,0.0,NaN,0.3,16.86,1.0,1.0,2.75,8.750000
2,0.000556,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,...,0.5,1.00,0.0,NaN,0.3,8.30,1.0,1.0,0.00,5.966667
3,0.000833,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,...,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.00,7.083333
7,0.001944,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,...,0.5,0.96,0.0,NaN,0.3,5.76,1.0,1.0,0.00,2.316667
